In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, \
                            recall_score, f1_score, log_loss

from tqdm.notebook import tqdm

from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

import joblib


RAND=42
N_FOLDS = 5

In [2]:
def get_metrics(y_test, y_pred, y_score, name = "Default"):
    """Метрики для задачи классификации"""
    df_metrics = pd.DataFrame()

    df_metrics['model'] = [name]
    df_metrics['Accuracy'] = accuracy_score(y_test, y_pred)
    df_metrics['ROC_AUC'] = roc_auc_score(y_test, y_score[:, 1])
    df_metrics['Precision'] = precision_score(y_test, y_pred)
    df_metrics['Recall'] = recall_score(y_test, y_pred)
    df_metrics['f1'] = f1_score(y_test, y_pred)
    df_metrics['Logloss'] = log_loss(y_test, y_score)

    return df_metrics

# Feature engineering

In [3]:
filepath = "../data/transformed/train_data/train_data_0.pq.0"
X_0 = pd.read_parquet(filepath).set_index("id")
X_0[:5]

,enc_paym_23_2,pre_maxover2limit_2,pre_loans6090_1,enc_loans_credit_status_5,pre_fterm_11,pre_loans_outstanding_2,enc_loans_credit_type_6,pre_till_pclose_1,is_zero_util_1,pre_loans90_13,...,enc_paym_1_3,enc_loans_account_holder_type_2,pre_maxover2limit_11,pre_till_pclose_5,pre_since_opened_0,pre_since_opened_11,pre_since_opened_14,enc_paym_17_2,pre_pterm_16,rn
id,,,,,,,,,,,,,,,,,,,,,
0,0,0,0,0,0,1,0,1,6,0,...,0,0,0,0,0,0,0,0,0,10
1,0,0,0,0,1,2,0,1,10,0,...,3,0,0,0,0,0,3,0,0,14
2,0,0,0,0,0,1,0,2,1,0,...,1,0,1,0,0,0,0,0,0,3
3,0,0,0,0,1,1,0,5,8,0,...,1,0,0,0,0,0,0,0,3,15
4,0,0,0,0,0,0,0,1,1,0,...,1,0,0,0,0,0,0,0,0,1


In [37]:
filepath = "../data/transformed/train_data/train_data_0.pq.1"
X_1 = pd.read_parquet(filepath).set_index("id")
X_1[:5]

,pre_since_confirmed_12,pre_loans3060_8,pre_loans_credit_limit_17,pre_since_confirmed_1,enc_loans_account_holder_type_6,pre_loans5_6,is_zero_over2limit_0,pre_util_19,enc_paym_9_1,pre_loans5_13,...,pre_loans530_11,pre_maxover2limit_1,pclose_flag_1,enc_paym_0_0,enc_paym_22_1,is_zero_loans6090_1,enc_paym_8_1,pre_till_fclose_13,pre_over2limit_1,rn
id,,,,,,,,,,,,,,,,,,,,,
125000,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
125001,0,0,0,0,0,3,0,0,2,0,...,0,0,0,0,0,2,2,0,0,3
125002,0,0,0,0,0,9,0,0,0,0,...,0,0,1,0,0,9,0,0,0,9
125003,0,0,0,0,0,6,0,0,0,0,...,0,0,1,0,0,6,0,0,0,6
125004,0,0,0,0,0,5,0,0,0,0,...,0,0,1,0,0,5,0,0,0,5


In [38]:
X = pd.concat([X_0, X_1])

In [41]:
X.shape

(250000, 420)

In [39]:
target_path = "../data/raw/train_target.csv"
Y = pd.read_csv(target_path, nrows=250000, index_col="id")
Y[:5]

,flag
id,
0,0
1,0
2,0
3,0
4,0


In [40]:
Y.shape

(250000, 1)

In [42]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y, 
                                                    random_state=RAND)

# Baseline models

In [43]:
# Создаем и обучаем бейслайн LightGBM
model = LGBMClassifier(n_estimators=1000, class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


LGBMClassifier(class_weight='balanced', n_estimators=1000, random_state=42)

In [44]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [45]:
res = get_metrics(y_train, y_pred_tr, y_score_tr, name="lgbm_train")
res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))
res

/tmp/ipykernel_5516/429328251.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="lgbm_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.91342,0.987436,0.259622,0.971392,0.409735,0.238035
0,lgbm_test,0.87766,0.679152,0.079809,0.280543,0.124266,0.299737


In [46]:
ratio = y_train[y_train==0].shape[0] / y_train[y_train==1].shape[0]

# Создаем и обучаем бейслайн Catboost
model = CatBoostClassifier(scale_pos_weight=ratio, random_state=RAND, verbose=0)
model.fit(x_train, y_train)

In [47]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [48]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))
res

/tmp/ipykernel_5516/787895992.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, name="catboost_train"))
/tmp/ipykernel_5516/787895992.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, name="catboost_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.91342,0.987436,0.259622,0.971392,0.409735,0.238035
0,lgbm_test,0.87766,0.679152,0.079809,0.280543,0.124266,0.299737
0,catboost_train,0.97118,0.842539,0.997647,0.068531,0.128252,0.106406
0,catboost_test,0.96886,0.753300,0.083333,0.000646,0.001283,0.125227


In [49]:
model = RandomForestClassifier(random_state=RAND, verbose=0)
model.fit(x_train, y_train)

/tmp/ipykernel_5516/1478570180.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)


RandomForestClassifier(random_state=42)

In [50]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [51]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))
res

/tmp/ipykernel_5516/3198375833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
/tmp/ipykernel_5516/3198375833.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.913420,0.987436,0.259622,0.971392,0.409735,0.238035
0,lgbm_test,0.877660,0.679152,0.079809,0.280543,0.124266,0.299737
0,catboost_train,0.971180,0.842539,0.997647,0.068531,0.128252,0.106406
0,catboost_test,0.968860,0.753300,0.083333,0.000646,0.001283,0.125227
0,LG_train,0.999465,0.999992,0.998851,0.983837,0.991287,0.025353
0,LG_test,0.968860,0.705033,0.222222,0.002586,0.005112,0.200392


In [52]:
# Создаем и обучаем линейную регрессию
model = LogisticRegression(class_weight='balanced', random_state=RAND)
model.fit(x_train, y_train)

/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(class_weight='balanced', random_state=42)

In [53]:
y_pred_tr = model.predict(x_train)
y_score_tr = model.predict_proba(x_train)

y_pred = model.predict(x_test)
y_score = model.predict_proba(x_test)

In [54]:
res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))
res

/tmp/ipykernel_5516/3198375833.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_train, y_pred_tr, y_score_tr, "LG_train"))
/tmp/ipykernel_5516/3198375833.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  res = res.append(get_metrics(y_test, y_pred, y_score, "LG_test"))


,model,Accuracy,ROC_AUC,Precision,Recall,f1,Logloss
0,lgbm_train,0.913420,0.987436,0.259622,0.971392,0.409735,0.238035
0,lgbm_test,0.877660,0.679152,0.079809,0.280543,0.124266,0.299737
0,catboost_train,0.971180,0.842539,0.997647,0.068531,0.128252,0.106406
0,catboost_test,0.968860,0.753300,0.083333,0.000646,0.001283,0.125227
0,LG_train,0.999465,0.999992,0.998851,0.983837,0.991287,0.025353
0,LG_test,0.968860,0.705033,0.222222,0.002586,0.005112,0.200392
0,LG_train,0.692410,0.761734,0.068636,0.711492,0.125196,0.592374
0,LG_test,0.687540,0.746294,0.065984,0.691661,0.120475,0.600030


**Выводы**  
Среди выбранных baseline-моделей catboost показал большое переобучение, что можно нивилировать, подобрав гиперпараметры; lightgbm показал меньше переобучение и большое значение ROC-AUC на валидации; логистическая регрессия показала слишком низкие метрики и на трейне и на валидации, что может говорить о том, что линейная модель не подходит для данной задачи.
  
Так как в нашем случае матрица объект-признаки имеет большую размерность, lightgbm является более подходящей моделью для обучения на стационарном устройстве, так как сам алгоритм работает быстрее.

# Tune

In [8]:
import optuna
import scipy.stats as stats

In [9]:
def objective_gpt(trial: optuna.Trial, x, y):
    params = {
        'n_estimators': trial.suggest_categorical('n_estimators', [1000]),
#         'learning_rate': trial.suggest_float('learning_rate', 0.001, 3),
        'learning_rate': trial.suggest_categorical('learning_rate', [0.01752291849636367]),
        'max_bin': trial.suggest_int('max_bin', 10, 500),
        'num_leaves': trial.suggest_int('num_leaves', 10, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 15),
        'min_child_samples': trial.suggest_int('min_child_samples', 100, 70000, step=100),
        'lambda_l1': trial.suggest_int('lambda_l1', 0, 100),
        'lambda_l2': trial.suggest_int('lambda_l2', 0, 100),
        'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
        'objective': trial.suggest_categorical('objective', ['binary']),
        'metric': trial.suggest_categorical('metric', ['auc']),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'random_state': trial.suggest_categorical('random_state', [RAND]),
    }

    cv_pred = np.empty(N_FOLDS)
    cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)

    for fold, (train_idx, test_idx) in enumerate(cv.split(x, y)):
        x_train_, x_val_ = x[train_idx, :], x[test_idx, :]
        y_train_, y_val_ = y[train_idx], y[test_idx]

#         ratio = y_train_[y_train_ == 0].shape[0] / \
#             y_train_[y_train_ == 1].shape[0]

        pruning = optuna.integration.LightGBMPruningCallback(trial, 'auc')

        model = LGBMClassifier(
            class_weight="balanced",
            n_jobs=-1,
            **params
        )
        model.fit(x_train_, y_train_,
                  eval_metric='auc',
                  eval_set=[(x_val_, y_val_)],
                  early_stopping_rounds=100,
                  callbacks=[pruning],
                  verbose=0)

        y_pred = model.predict(x_val_)
        y_proba = model.predict_proba(x_val_)[:, 1]

        cv_pred[fold] = roc_auc_score(y_val_, y_proba)
    return (np.mean(cv_pred))

In [10]:
func = lambda trial: objective_gpt(trial, x_train, y_train.values)

study = optuna.create_study(direction="maximize")
study.optimize(func, n_trials=10, show_progress_bar=True, n_jobs=4)

[I 2023-06-07 19:11:33,709] A new study created in memory with name: no-name-46a24ead-ed5b-4d9b-8764-8dec1a3d0a97
/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_49385/2842095603.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_49385/2842095603.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_49385/2842095603.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/tmp/ipyk

[LightGBM] [Warning] lambda_l1 is set=39, reg_alpha=0.0 will be ignored. Current value: lambda_l1=39
[LightGBM] [Warning] bagging_fraction is set=0.3566144657555625, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3566144657555625
[LightGBM] [Warning] [LightGBM] [Warning] feature_fraction is set=0.677110046735162, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.677110046735162
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6lambda_l1 is set=15, reg_alpha=0.0 will be ignored. Current value: lambda_l1=15

[LightGBM] [Warning] lambda_l2 is set=23, reg_lambda=0.0 will be ignored. Current value: lambda_l2=23
[LightGBM] [Warning] bagging_fraction is set=0.9837031837329511, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9837031837329511
[LightGBM] [Warning] feature_fraction is set=0.8968525103799552, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.89685

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=39, reg_alpha=0.0 will be ignored. Current value: lambda_l1=39
[LightGBM] [Warning] bagging_fraction is set=0.3566144657555625, subsample=1.0 will be ignored. Current value: bagging_fraction=0.3566144657555625
[LightGBM] [Warning] feature_fraction is set=0.677110046735162, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.677110046735162
[LightGBM] [Warning] bagging_freq is set=6, subsample_freq=0 will be ignored. Current value: bagging_freq=6
[LightGBM] [Warning] lambda_l2 is set=23, reg_lambda=0.0 will be ignored. Current value: lambda_l2=23


/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 0 is already reported.
  warnings.warn(
/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/trial/_trial.py:490: UserWarning: The reported value is ignored because this `step` 1 is already reported.
  warnings.warn(
/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/trial/_trial.py:490: User

limit_output extension: Maximum message size of 10000 exceeded with 13812 characters

In [ ]:
study.optimize(func, n_trials=10, show_progress_bar=True, n_jobs=4)

/home/sergey/virtual_env/lib/python3.10/site-packages/optuna/progress_bar.py:56: ExperimentalWarning: Progress bar is experimental (supported from v1.2.0). The interface can change in the future.
  self._init_valid()


  0%|          | 0/10 [00:00<?, ?it/s]

/tmp/ipykernel_49385/2842095603.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_49385/2842095603.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_49385/2842095603.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/tmp/ipyk

[LightGBM] [Warning] lambda_l1 is set=43, reg_alpha=0.0 will be ignored. Current value: lambda_l1=43
[LightGBM] [Warning] bagging_fraction is set=0.20668429949194433, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20668429949194433
[LightGBM] [Warning] feature_fraction is set=0.2129898585990041, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.2129898585990041
[LightGBM] [Warning] bagging_freq is set=1, subsample_freq=0 will be ignored. Current value: bagging_freq=1
[LightGBM] [Warning] lambda_l2 is set=31, reg_lambda=0.0 will be ignored. Current value: lambda_l2=31
[LightGBM] [Warning] lambda_l1 is set=4, reg_alpha=0.0 will be ignored. Current value: lambda_l1=4
[LightGBM] [Warning] bagging_fraction is set=0.20546518574674189, subsample=1.0 will be ignored. Current value: bagging_fraction=0.20546518574674189
[LightGBM] [Warning] feature_fraction is set=0.5399119019882473, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5

/tmp/ipykernel_49385/2842095603.py:12: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'min_split_gain': trial.suggest_loguniform('min_split_gain', 1e-4, 1e-1),
/tmp/ipykernel_49385/2842095603.py:15: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'feature_fraction': trial.suggest_uniform('feature_fraction', 0.2, 1.0),
/tmp/ipykernel_49385/2842095603.py:16: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.2, 1.0),
/home/ser

limit_output extension: Maximum message size of 10000 exceeded with 10346 characters

In [20]:
study.best_params

{'n_estimators': 1000, 'learning_rate': 0.01752291849636367}

In [27]:
# Посчитаем микро- и макро- усреднение roc-auc 
cv = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=RAND)
roc_aucs = []
y_pred_hold = []
y_score_hold = []

for fold, (train_idx, test_idx) in enumerate(cv.split(x_train, y_train)):
    x_train_, x_val_ = x_train[train_idx, :], x_train[test_idx, :]
    y_train_, y_val_ = y_train.values[train_idx], y_train.values[test_idx]

#     ratio = y_train_[y_train_ == 0].shape[0] / \
#         y_train_[y_train_ == 1].shape[0]

    model = LGBMClassifier(**study.best_params, class_weight='balanced', n_jobs=-1)
    model.fit(x_train_, y_train_,
              eval_metric='auc',
              eval_set=[(x_val_, y_val_)],
              early_stopping_rounds=100,
              verbose=0)

    y_score = model.predict_proba(x_val_)

    roc_aucs.append(roc_auc_score(y_val_, y_score[:, 1]))

    y_pred_hold.append(model.predict(x_test))
    y_score_hold.append(model.predict_proba(x_test))

y_pred = stats.mode(np.column_stack(y_pred_hold), axis=1)[0]
y_score = np.column_stack([np.mean(np.column_stack(y_score_hold)[:, ::2], axis=1),
                           np.mean(np.column_stack(y_score_hold)[:, 1::2], axis=1)])

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7635229198560924, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7635229198560924
[LightGBM] [Warning] feature_fraction is set=0.7697099444692574, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7697099444692574
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7635229198560924, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7635229198560924
[LightGBM] [Warning] feature_fraction is set=0.7697099444692574, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7697099444692574
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...

/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7635229198560924, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7635229198560924
[LightGBM] [Warning] feature_fraction is set=0.7697099444692574, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7697099444692574
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85


/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/home/sergey/virtual_env/lib/python3.10/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] lambda_l1 is set=0, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0
[LightGBM] [Warning] bagging_fraction is set=0.7635229198560924, subsample=1.0 will be ignored. Current value: bagging_fraction=0.7635229198560924
[LightGBM] [Warning] feature_fraction is set=0.7697099444692574, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7697099444692574
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
[LightGBM] [Warning] lambda_l2 is set=85, reg_lambda=0.0 will be ignored. Current value: lambda_l2=85


/tmp/ipykernel_35264/2488522396.py:28: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  y_pred = stats.mode(np.column_stack(y_pred_hold), axis=1)[0]


In [28]:
print(np.mean(roc_aucs))
print(roc_auc_score(y_test, y_score[:, 1]))

0.756945243801509
0.7553409536969106


In [29]:
params = study.best_params

joblib.dump(params, "../data/params.joblib")

['../data/params.joblib']

# Out-of-core training

In [4]:
from scrypt import out_of_core_train

In [5]:
train_path = "../data/transformed/train_data/"
target_path = "../data/raw/train_target.csv"

# params = joblib.load("../data/params.joblib")

In [6]:
model = out_of_core_train(train_path, target_path)

/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


  0%|          | 0/11 [00:00<?, ?it/s]

/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/sergey/virtual_env/lib/python3.10/site-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was

In [7]:
# Создаем словарь с моделью и MinMaxScaler'ом
models = {}

models['lightgbm'] = model

In [7]:
# Сохраняем словарь с моделями в joblib файл
models_path = '../data/models.joblib'
joblib.dump(models, models_path)

['../data/models.joblib']